In [ ]:
import json 
import pickle 
import nltk
import numpy as np

from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from nltk.stem import SnowballStemmer
import random


CONSTRUCCIÓN DE LA BIBLIOTECA DE INTENCIONES 

In [ ]:
def guardar_json(datos,filename):
    archivo=open(filename,'w')
    json.dump(datos,archivo,indent=4)

In [ ]:
biblioteca = {"intents": 
              
              [{"tag":"saludos",
                 "patterns":['hola',
                            'buenos dias',
                            'buenas tardes',
                            'como estas?',
                            'tengo una consulta',
                            'necesito ayuda',
                            'que me puedes ofrecer?',
                            'que haces?'],

                 "responses":['''Hola, soy Rockie!! un asistente virtual, en qué puedo ayudarte? 
                                 Si desea conocer lo que puedo ofrecer envíe *1* '''],

                 "context":[""]
                 },    
                 
                 
                 
                 {"tag":"despedidas",
                 "patterns":['chao','adios','hasta luego','nos vemos','hasta la proxima'],
                "responses":['Hasta luego que tenga un buen dia, gracias por preferirnos!'],
                "context":[],
                 },
                 
                
                 {"tag":"agradecimientos",
                 "patterns":["gracias",
                             "muchas gracias",
                             "mil gracias",
                             "muy amable",
                             "se lo agradezco",
                             "fue de ayuda",
                             "gracias por la ayuda",
                             "muy agradecido",
                             "gracias por su tiempo"                             
                            ],

                 "responses":["De nada",
                              "Feliz por ayudarlo",
                              "Gracias a usted",
                              "Estamos para servirle",
                              "Fue un placer",
                              "Un gusto comunicarme contigo"
                             ],
                 "context":[""]
                },
                 
                 
                 {"tag":"Agendar cita",
                 "patterns":["Quisiera agendar una cita",
                             "Puedo agendar una cita?",
                             'Quiero agendar cita',
                             "Quiero agendar una cita"                          
                            ],
                 "responses":["""Para agendar una cita indique el servicio que desea:
                 
                                1. Asesoría de compras
                                2. Atención al cliente
                                 """
                             ],
                 "context":[""]
                },
                 
               
                {"tag":"Modificar Cita",
                 "patterns":["Quiero modificar mi cita",
                             "Puedo modificar mi cita?",
                             "Modificar cita"                          
                            ],
                 "responses":["""Para modificar su cita ingrese el id dado al momento de generar su cita.
                                  """
                             ],
                 "context":[""]
                },
               
               
               
                {"tag":"Cancelar Cita",
                 "patterns":["Quiero cancelar mi cita",
                             "Puedo cancelar mi cita?",
                             "Cancelar cita"                          
                            ],
                 "responses":[""" Para modificar su cita ingrese el id de cita dado al momento de generar su cita
                                  """
                             ],
                 "context":[""]
                },
               
               
               
                 
                 {"tag":"Ubicación",
                 "patterns":["Ubicación", 
                             "Dónde se ubican?",
                             "Tienen un local físico?"
                            ],
                 "responses":[""" Estamos ubicado en Costa del Este, Ciudad de Panamá, Panamá
                                   Aquí el link de nuestra ubicación exacta: 
                                  """
                             ],
                 "context":[""]
                },
                 
                 
                {"tag":"norespuesta",
                 "patterns":[""],
                 "responses":["No puedo reponder a tu consulta, podrías ser más claro?",
                              "Disculpa no entiendo tu solicitud, podrías ser más claro?"
                             ],
                 "context":[""]                    
                },

                {
                    "tag": "Servicio",
                    "patterns": [
                        "Que tipo de servicio ofrecen?",
                        "Servicios",
                        "Que hacen?",
                        'A qué se dedican'
                    ],
                    "responses": [
                        """Nuestra empresa SIC-Panamá se encarga de transportar cualquier producto que necesites a nivel mundial, 
                            para bridarle a nuestros clientes contamos con un conjunto de herramientas que le permita escoger la 
                            opción que más se adaptan a ellos, como asesorías directa o online,cotización de precios y mucho más"""
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "Confrontación con el chatbot",
                    "patterns": [
                        "Eres un chatbot",
                        "Estoy escribiendole a un agente virtual?",
                        "Eres una persona real?"
                    ],
                    "responses": [
                    "Soy un agente virtual construido con tecnología"
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "Cotización de precios",
                    "patterns": [
                        "Quiero cotizar precios",
                        "Deseo cotizar precios",
                        "Cotizar precios"
                    ],
                    "responses": [
                        "Para cotizar precios, indiquenos el paquete que desea. Si desea conocer cuáles osn envíe *1*"
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "Menú de opciones",
                    "patterns": [
                        ""
                    ],
                    "responses": [
                        """1. Cita de asesoría
                           2. Cotizar precios
                           3. Envío de paquetes"""
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "",
                    "patterns": [
                        "",
                        "",
                        ""
                    ],
                    "responses": [
                        "....."
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "",
                    "patterns": [
                        "",
                        "",
                        ""
                    ],
                    "responses": [
                        "....."
                    ],
                    "context": [
                        ""
                    ]
                }
                ]            
                }


# Guardado de diccionario de conocimiento en formato json.
guardar_json(biblioteca, 'intents.json')
biblioteca

In [ ]:
bolsaPalabras=[]
clases=[]
documentos=[]

for intent in biblioteca['intents']:

    clases.append(intent['tag'])


    for pattern in intent['patterns']:
        result=nltk.word_tokenize(pattern)
        bolsaPalabras.extend(result)
        documentos.append((result,intent['tag']))

print(bolsaPalabras)


In [ ]:
stemmer=SnowballStemmer('spanish')

ignore_words= ["?","¿","!","¡","."] 

bolsaPalabras2=[]

for w in bolsaPalabras:
    if w not in ignore_words:
        wprocesada=w.lower()
        wprocesada=stemmer.stem(wprocesada)

print('bolsaPalabra2:',bolsaPalabras2)

 OBTENER DATOS DE ENTRENAMIENTO 

In [ ]:
def cleanString(words,ignore_words):
    words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    return words

bolsaPalabras=cleanString(bolsaPalabras,ignore_words)
pickle.dump(bolsaPalabras, open('bolsaPalabras.pkl','wb'))
training= []

for doc in documentos:

    interaccion=doc[0]
    interaccion=cleanString(interaccion,ignore_words)

    entradacodificada=[]

    for palabra in bolsaPalabras:
        if palabra in interaccion:
            entradacodificada.append(1)
        else:
            entradacodificada.append(0)

    salidacodificada=[0]*len(clases)
    indice=clases.index(doc[1])
    salidacodificada[indice]=1

    training.append([entradacodificada,salidacodificada])

training=np.array(training,dtype=list)
x_train=list(training[:,0])
y_train=list(training[:,1])


CREAR REDES NEURONALES

In [ ]:
model=Sequential()

model.add(Dense(128,input_shape=(len(x_train[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

ENTRENAR MODELO

In [ ]:
sgd=SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accurracy'])
hist=model.fit(np.array(x_train),epochs=300,batch_size=5,verbose=True)
model.save('bot_model.h5',hist)



###CORRER DESPUÉS DE COLOCAR TODAS LAS INTENCIONES 

PREPARAR FUNCIONES DE RESPUESTAS DEL CHATBOT

In [ ]:
ignore_words= ["?","¿","!","¡","."]
model=load_model('bot_model.h5')
biblioteca=json.loads(open('intents.json').read())
bolsaPalabras=pickle.load(open('bolsaPalabra.pkl','rb'))
clases=pickle.load(open('classes.pkl','rb'))

In [ ]:
def limpiarEntrada(entradaUsuario):
    entradaUsuario=nltk.word_tokenize(entradaUsuario)
    entradaUsuario=[stemmer.stem(w.lower()) for w in entradaUsuario if w not in ignore_words]
    return entradaUsuario

def convVector(entradaUsuario,bolsaPalabras):

    entradaUsuario=limpiarEntrada(entradaUsuario)
    
    vector_entrada=[0]*len(bolsaPalabras)

    for palabras in entradaUsuario:
        if palabra in bolsaPalabras:
          omdoce=bolsaPalabras.index(palabra)
          vector_entrada[indice]=1

        vector_entrada=np.array(vector_entrada)
        return vector_entrada


In [ ]:
def obt_tag(vectorentrada,LIMITE=0):
    vector_salida=model.predict(np.array([vectorentrada]))[0]

    vector_salida=[[i,r] for i,r in enumerate(vector_salida) if r > LIMITE]

    vector_salida.sort(key=lambda x: x[1],reverse=True)
    print(vector_salida)

    list_etiquetas=[]

    for r in vector_salida:
        list_etiquetas.append({'intent':clases[r[0]],'probability':str(r[1])})
   
    return list_etiquetas



In [ ]:

def obt_respuesta(list_etiquetas,biblioteca):
    etiqueta=list_etiquetas[0]['intent']

    lista_diccionarios=biblioteca['intents']

    for diccionario in lista_diccionarios:

        if etiqueta==diccionario['tag']:
            list_respuestas=diccionario['responses']
            respuesta=random.choice(list_respuestas)

            break
    
    return respuesta


In [ ]:
def chatbot_respuesta(entradaUsuario):
    vector_entrada=convVector(entradaUsuario,bolsaPalabras)
    list_etiquetas=obt_tag(vector_entrada,LIMITE=0)
    respuesta=obt_respuesta(list_etiquetas,biblioteca)
    intencion=list_etiquetas[0]['intent']
    return respuesta,intencion